### Tóm tắt bằng phương pháp trừu tượng

In [1]:
!pip install torch
!pip install transformers
!pip install datasets

In [2]:
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk import download
download('stopwords')

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# Đọc dữ liệu từ các tệp CSV
train_data = pd.read_csv('/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv')
test_data = pd.read_csv('/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv')
val_data = pd.read_csv('/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv')

In [4]:
train_data=train_data.head(20000)
val_data=val_data.head(20000)

In [5]:
# Xóa cột không cần thiết (id)
train_data = train_data.drop('id', axis=1)
test_data = test_data.drop('id', axis=1)
val_data = val_data.drop('id', axis=1)

In [6]:
# Tiền xử lý dữ liệu văn bản
def preprocess_text(text):
    # Loại bỏ ký tự đặc biệt
    text = re.sub(r'[^\w\s]', '', text)
    # Chuyển đổi chữ hoa thành chữ thường
    text = text.lower()
    # Loại bỏ stopwords (từ không có ý nghĩa)
    stopwords_list = set(stopwords.words('english'))
    words = text.split()
    words = [word for word in words if word not in stopwords_list]
    # Ghép các từ lại thành câu
    text = ' '.join(words)
    return text

train_data['article'] = train_data['article'].apply(preprocess_text)
train_data['highlights'] = train_data['highlights'].apply(preprocess_text)

val_data['article'] = val_data['article'].apply(preprocess_text)
val_data['highlights'] = val_data['highlights'].apply(preprocess_text)

# test_data['article'] = test_data['article'].apply(preprocess_text)
# test_data['highlights'] = test_data['highlights'].apply(preprocess_text)

In [7]:
train_data

,article,highlights
0,associated press published 1411 est 25 october...,bishop john folda north dakota taking time dia...
1,cnn ralph mata internal affairs lieutenant mia...,criminal complaint cop used role help cocaine ...
2,drunk driver killed young woman headon crash c...,craig ecclestontodd 27 drunk least three pints...
3,cnn breezy sweep pen president vladimir putin ...,nina dos santos says europe must ready accept ...
4,fleetwood team still 100 record sky bet league...,fleetwood top league one 20 win scunthorpe pet...
...,...,...
19995,daily mail reporter published 1827 est 1 octob...,flight turned around way miami monday
19996,simon tomlinson decade abuse yemeni woman noor...,noora al shamis parents gave distant cousin es...
19997,say manchester city crystal palace teams movin...,man city crystal palace arrived manchester pic...
19998,manchester united captain wayne rooney cheated...,wayne rooney cheated preston believes stan col...


In [8]:
max_input_length = 512
max_target_length = 100

prefix = "summarize: "
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["highlights"], max_length=max_target_length, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [9]:
from datasets import Dataset
train_ds = Dataset.from_pandas(train_data)
test_ds = Dataset.from_pandas(test_data)
val_ds = Dataset.from_pandas(val_data)

In [10]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-base")

train_ds = train_ds.map(preprocess_function, batched=True)
val_ds = val_ds.map(preprocess_function, batched=True)
test_ds = test_ds.map(preprocess_function, batched=True)

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


  0%|          | 0/20 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

In [11]:
train_ds

Dataset({
    features: ['article', 'highlights', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 20000
})

In [12]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.nn import CrossEntropyLoss
from transformers import AdamW
from torch.nn.utils.rnn import pad_sequence
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-base")

# Chuẩn bị dữ liệu
class TextDataset(Dataset):
    def __init__(self, data):
        self.input_ids = data['input_ids']
        self.attention_mask = data['attention_mask']
        self.labels = data['labels']

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, index):
        return {
            'input_ids': torch.tensor(self.input_ids[index]),
            'attention_mask': torch.tensor(self.attention_mask[index]),
            'labels': torch.tensor(self.labels[index])
        }

def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    labels = [item['labels'] for item in batch]

    input_ids = pad_sequence(input_ids, batch_first=True)
    attention_mask = pad_sequence(attention_mask, batch_first=True)
    labels = pad_sequence(labels, batch_first=True)

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels
    }

train_dataset = TextDataset(train_ds)
train_dataloader = DataLoader(train_dataset,batch_size=12, shuffle=True,collate_fn=collate_fn)

test_dataset = TextDataset(test_ds)
test_dataloader = DataLoader(test_dataset, shuffle=True,collate_fn=collate_fn)

val_dataset = TextDataset(val_ds)
val_dataloader = DataLoader(val_dataset, batch_size=12,shuffle=True,collate_fn=collate_fn)

In [26]:
len(train_dataloader)

1667

In [13]:
# Tạo mô hình T5
model = T5ForConditionalGeneration.from_pretrained('t5-base')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Huấn luyện mô hình
optimizer = AdamW(model.parameters(), lr=0.0005)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [14]:
epochs = 10

for epoch in range(epochs):
    total_loss = 0
    for batch in train_dataloader:
      input_ids = pad_sequence(batch['input_ids'], batch_first=True)
      attention_mask = pad_sequence(batch['attention_mask'], batch_first=True)
      labels = pad_sequence(batch['labels'], batch_first=True)

      inputs = {
          'input_ids': input_ids,
          'attention_mask': attention_mask,
          'labels': labels
      }

      inputs = {key: value.to(device) for key, value in inputs.items()}

      outputs = model(**inputs)
      loss = outputs.loss

      # Backward propagation and optimization
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      total_loss += loss.item()
    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")


Epoch 1/10 - Loss: 1.4312
Epoch 2/10 - Loss: 1.1243
Epoch 3/10 - Loss: 0.8598
Epoch 4/10 - Loss: 0.6074
Epoch 5/10 - Loss: 0.4138
Epoch 6/10 - Loss: 0.2704
Epoch 7/10 - Loss: 0.1781
Epoch 8/10 - Loss: 0.1332
Epoch 9/10 - Loss: 0.1118
Epoch 10/10 - Loss: 0.0953


In [15]:
# Đánh giá trên tập dữ liệu val
model.eval()  
val_loss = 0

with torch.no_grad():
    for val_batch in val_dataloader:
        val_input_ids = pad_sequence(val_batch['input_ids'], batch_first=True)
        val_attention_mask = pad_sequence(val_batch['attention_mask'], batch_first=True)
        val_labels = pad_sequence(val_batch['labels'], batch_first=True)

        val_inputs = {
            'input_ids': val_input_ids,
            'attention_mask': val_attention_mask,
            'labels': val_labels
        }

        val_inputs = {key: value.to(device) for key, value in val_inputs.items()}

        val_outputs = model(**val_inputs)
        val_loss += val_outputs.loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Validation Loss: {avg_val_loss:.4f}")



Validation Loss: 2.7905


In [16]:
# Tạo tóm tắt
text = "cnn ralph mata internal affairs lieutenant miamidade police department working division investigates allegations wrongdoing cops outside office authorities allege 45yearold longtime officer worked drug trafficking organization help plan murder plot get guns criminal complaint unsealed us district court new jersey tuesday accuses mata also known milk man using role police officer help drug trafficking organization exchange money gifts including rolex watch one instance complaint alleges mata arranged pay two assassins kill rival drug dealers killers would pose cops pulling targets shooting according complaint ultimately organization decided move forward murder plot mata still received payment setting meetings federal prosecutors said statement complaint also alleges mata used police badge purchase weapons drug traffickers mata according complaint used contacts airport transport weapons carryon luggage trips miami dominican republic court documents released investigators specify name drug trafficking organization mata allegedly conspired says organization importing narcotics places ecuador dominican republic hiding inside shipping containers containing pallets produce including bananas organization distributing narcotics new jersey elsewhere complaint says authorities arrested mata tuesday miami gardens florida immediately clear whether mata attorney police officials could immediately reached comment mata worked miamidade police department since 1992 including directing investigations miami gardens working lieutenant k9 unit miami international airport according complaint since march 2010 working internal affairs division mata faces charges aiding abetting conspiracy distribute cocaine conspiring distribute cocaine engaging monetary transactions property derived specified unlawful activity scheduled appear federal court florida wednesday convicted mata could face life prison cnns suzanne presto contributed report"
inputs = tokenizer.encode_plus(text, return_tensors='pt', max_length=512, truncation=True)
input_ids = inputs['input_ids'].to(device)
attention_mask = inputs['attention_mask'].to(device)

output = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=50)

summary = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"Summary: {summary}")

Summary: criminal complaint cop used role help cocaine traffickers ralph mata internal affairs lieutenant allegedly helped group get guns also arranged pay two assassins murder rivals complaint alleges mata milk man used police badge


In [19]:
model.save_pretrained("directory")


In [23]:
!pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [25]:
from rouge import Rouge

In [29]:
# Tạo dự đoán
text = test_data['article'][0]
inputs = tokenizer.encode_plus(text, return_tensors='pt', max_length=512, truncation=True)
input_ids = inputs['input_ids'].to(device)
attention_mask = inputs['attention_mask'].to(device)

output = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=50)

summary = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"Summary: {summary}")

Summary: group consumer advocacy group says having 31 inches space on planes isnt squabbling over the arm rest inconsiderate passengers on board could risk taking hours removing air space on planes experts say putting people risk also 


In [31]:
rouge = Rouge()
scores = rouge.get_scores(summary, test_data['highlights'][0], avg=True)

# Print the ROUGE scores
print(f"ROUGE-1: {scores['rouge-1']['f']:.4f}")
print(f"ROUGE-2: {scores['rouge-2']['f']:.4f}")
print(f"ROUGE-L: {scores['rouge-l']['f']:.4f}")

ROUGE-1: 0.2857
ROUGE-2: 0.0606
ROUGE-L: 0.2540


### Tóm tắt bằng phương pháp trích xuất

In [1]:
!pip install -U sentence-transformers

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
c:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model_v1 = SentenceTransformer('bert-base-nli-mean-tokens')
def tom_tat_v1(text, num_sentences,model_v1):
    sentences = sent_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokenzier = []
    for sentence in sentences:
        word_tokens = word_tokenize(sentence)
        tokenzie = [word for word in word_tokens if word.lower() not in stop_words]
        tokenzier.append(tokenzie)

    # Nhúng các câu
    sentence_embeddings = model_v1.encode(tokenzier)
    similarity_matrix = cosine_similarity(sentence_embeddings, sentence_embeddings)
    rank = sorted([(sentences[i], similarity_matrix.sum(axis=1)[i]) for i in range(len(sentences))], key=lambda x: x[1], reverse=True)
    summary_sentences = [sentence[0] for sentence in rank[:num_sentences]]
    summary = ' '.join(summary_sentences)

    return summary



In [ ]:
summary2 = tom_tat_v1(text, 5,model_v1)